<a href="https://colab.research.google.com/github/MWFK/NLP-Semantic-Similarity/blob/main/ClinicalTrials/Data%20Engineering/2-%20EligibilityCriteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
# pd.set_option('display.max_columns', None)  
# pd.set_option('display.max_colwidth', None)
import requests
from itertools import compress
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from google.colab import files

In [2]:
####### Search Expression #######
# None

####### Study Fields #######
# NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria 	

####### New Fields to add #######
# LocationStatus, Phase

####### Range Min_MAX ######
# 1 to 1000

####### Format #######
# CSV

url = 'https://clinicaltrials.gov/api/query/study_fields?expr=&fields=NCTId%2CBriefTitle%2CCondition%2CKeyword%2CEligibilityCriteria+%09&min_rnk=1&max_rnk=1000&fmt=csv'
session = requests.Session()
retry   = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://' , adapter)
session.mount('https://', adapter)

clinicaltrials = session.get(url)
print('Download Request Status: ', clinicaltrials.status_code)

csv_file = open('/content/'+str(1)+'-batch.csv', 'wb')
csv_file.write(clinicaltrials.content)
csv_file.close()

df = pd.read_csv(r'/content/1-batch.csv', skiprows=10)
print(df.shape)
df.head()

Download Request Status:  200
(1000, 6)


,Rank,NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria
0,1,NCT05237167,Time to Diagnosis of Glenohumeral Joint Disloc...,Glenohumeral Dislocation,glenohumeral dislocation|POC Ultrasound|Plain ...,Inclusion Criteria:||Adult patients (19 and ol...
1,2,NCT05237154,Intermittent Fasting and Exercise: Effects on ...,Composition and Function of the Gut Microbiota...,Obesity|Gut microbiota|Intermittent fasting|Hi...,Inclusion criteria:||Age between 18 and 40 yea...
2,3,NCT05237141,Etonogestrel Implant as Emergency Contraception,Contraception,NaN,Inclusion Criteria:||BMI less than or equal to...
3,4,NCT05237128,The Lifestyle Education About Nutrition for Di...,Type 2 Diabetes,Very low-carbohydrate diet|Moderate-carbohydra...,Inclusion Criteria:||diagnosis of type 2 diabe...
4,5,NCT05237115,Helicobacter Pylori Eradication With Probiotic...,Gastritis|Peptic Ulcer|Dyspepsia,Helicobacter pylori|Probiotics-combining tripl...,"Inclusion Criteria:||Age between 18~75,both ge..."


In [3]:
# Convert the EligibilityCriteria to string then lowercase
EligibilityCriteria = df['EligibilityCriteria'].astype(str).str.lower().tolist()
EligibilityCriteria[:10]

['inclusion criteria:||adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation||exclusion criteria:||injury sustained in major traumatic event (trauma activation), unable to consent, in extremis, less then 19 years of age',
 'inclusion criteria:||age between 18 and 40 years;|women;|have a regular menstrual cycle;|body weight below 120 kg;|bmi between 30 kg/m2 and 40 kg/m2.||exclusion criteria:||individuals with any disease;|smokers;|alcoholics;|use regular medication;|medical impediment to the practice of physical activities;|carry out nutritional monitoring;|being in treatment for weight loss;|present some discomfort in long periods of fasting.',
 'inclusion criteria:||bmi less than or equal to 28kg/m2|intact uterus with at least one ovary|regular menstrual cycles that occur every 21-35 days|if patient is postpartum or post-second trimester abortion, 3 menses (2 cycles) must have occurred p

In [4]:
# Seperate the InclusionCriteria from the ExclusionCriteria
InclusionCriteria = [txt[21:txt.find('exclusion criteria')-2] for txt in EligibilityCriteria]
InclusionCriteria[:10]

['adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation',
 'age between 18 and 40 years;|women;|have a regular menstrual cycle;|body weight below 120 kg;|bmi between 30 kg/m2 and 40 kg/m2.',
 'bmi less than or equal to 28kg/m2|intact uterus with at least one ovary|regular menstrual cycles that occur every 21-35 days|if patient is postpartum or post-second trimester abortion, 3 menses (2 cycles) must have occurred prior to enrollment|if patient had a first trimester abortion or pregnancy loss, she must have one spontaneous menses prior to enrollment|desires insertion of an etonogestrel contraceptive implant for contraception|not currently pregnant or trying to become pregnant|willing to abstain from medications and supplements known to induce/inhibit cyp3a4 during the study',
 'diagnosis of type 2 diabetes, with current hba1c > 6.5% and < 12% (self-reported and then measured at screening)|a

In [5]:
# Seperate the ExclusionCriteria from the InclusionCriteria
ExclusionCriteria = [txt[txt.find('exclusion criteria')+21:] for txt in EligibilityCriteria]
ExclusionCriteria[:10]

['injury sustained in major traumatic event (trauma activation), unable to consent, in extremis, less then 19 years of age',
 'individuals with any disease;|smokers;|alcoholics;|use regular medication;|medical impediment to the practice of physical activities;|carry out nutritional monitoring;|being in treatment for weight loss;|present some discomfort in long periods of fasting.',
 'have a known hypersensitivity or contraindications to etonogestrel.|medical conditions that affect liver function (e.g., hepatitis, cirrhosis; assessed via participant self-report)|known or suspected current alcohol dependence syndrome or any illicit drug use that may affect the metabolism of the etonogestrel.|uncontrolled thyroid disorder.|use of long-acting injectable hormonal contraceptive within the past 9 months|current use of hormonal oral, patch, intravaginal, or intrauterine contraception',
 "unable to provide informed consent|substance abuse, mental health, cognitive, or medical condition that wou

In [6]:
# Split each Trial InclusionCriteria into list of lists of the InclusionCriteria
InclusionCriteriaList = list(map(lambda txt : txt.split("|"), InclusionCriteria))
InclusionCriteriaList = [list(filter(lambda txt: txt!= '', ltexts)) for ltexts in InclusionCriteriaList] # Delete the empty string generated by successive || (sometimes they seperate with | or ||)
InclusionCriteriaList[:2]

[['adult patients (19 and older in state of nebraska) who present to the emergency department with shoulder pain/injury and potential shoulder dislocation'],
 ['age between 18 and 40 years;',
  'women;',
  'have a regular menstrual cycle;',
  'body weight below 120 kg;',
  'bmi between 30 kg/m2 and 40 kg/m2.']]

In [7]:
# Split each Trial ExclusionCriteria into list of lists of the ExclusionCriteria
ExclusionCriteriaList = list(map(lambda txt : txt.split("|"), ExclusionCriteria))
ExclusionCriteriaList = [list(filter(lambda txt: txt!= '', ltexts)) for ltexts in ExclusionCriteriaList] # Delete the empty string generated by successive || (sometimes they seperate with | or ||)
ExclusionCriteriaList[:2]

[['injury sustained in major traumatic event (trauma activation), unable to consent, in extremis, less then 19 years of age'],
 ['individuals with any disease;',
  'smokers;',
  'alcoholics;',
  'use regular medication;',
  'medical impediment to the practice of physical activities;',
  'carry out nutritional monitoring;',
  'being in treatment for weight loss;',
  'present some discomfort in long periods of fasting.']]

In [8]:
df['InclusionCriteriaList'] = InclusionCriteriaList
df['ExclusionCriteriaList'] = ExclusionCriteriaList
df.head()

,Rank,NCTId,BriefTitle,Condition,Keyword,EligibilityCriteria,InclusionCriteriaList,ExclusionCriteriaList
0,1,NCT05237167,Time to Diagnosis of Glenohumeral Joint Disloc...,Glenohumeral Dislocation,glenohumeral dislocation|POC Ultrasound|Plain ...,Inclusion Criteria:||Adult patients (19 and ol...,[adult patients (19 and older in state of nebr...,[injury sustained in major traumatic event (tr...
1,2,NCT05237154,Intermittent Fasting and Exercise: Effects on ...,Composition and Function of the Gut Microbiota...,Obesity|Gut microbiota|Intermittent fasting|Hi...,Inclusion criteria:||Age between 18 and 40 yea...,"[age between 18 and 40 years;, women;, have a ...","[individuals with any disease;, smokers;, alco..."
2,3,NCT05237141,Etonogestrel Implant as Emergency Contraception,Contraception,NaN,Inclusion Criteria:||BMI less than or equal to...,"[bmi less than or equal to 28kg/m2, intact ute...",[have a known hypersensitivity or contraindica...
3,4,NCT05237128,The Lifestyle Education About Nutrition for Di...,Type 2 Diabetes,Very low-carbohydrate diet|Moderate-carbohydra...,Inclusion Criteria:||diagnosis of type 2 diabe...,"[diagnosis of type 2 diabetes, with current hb...","[unable to provide informed consent, substance..."
4,5,NCT05237115,Helicobacter Pylori Eradication With Probiotic...,Gastritis|Peptic Ulcer|Dyspepsia,Helicobacter pylori|Probiotics-combining tripl...,"Inclusion Criteria:||Age between 18~75,both ge...","[age between 18~75,both gender., patients with...",[patients are excluded if they have previously...


In [9]:
df.to_excel('df.xlsx')
files.download('df.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>